# Project 2 - Polynomial Models for Boston House Prices

## Group members
   Michael Peralta (mikeperalta@csu.fullerton.edu)
   
   Brandon Xue (brandonx@csu.fullerton.edu)

### CPSC 483-02, Section ID: 33018

# Experiment 1
Load and examine the Boston dataset’s features, target values, and description.

In [ ]:
# Reset the environment variables without confirmation.
# This avoids broken code from functioning off of artifacts
# left behind from previous executions.
%reset -f

# 1. Load and examine
import pandas as pd
import numpy as np
from sklearn.datasets import *

bs = load_boston()
bs.DESCR # description
bs.data # features
bs.target # targets
print("Features: ", end="")
print(*bs.feature_names, sep=", ")

# Experiment 2
Use *sklearn.model_selection.train_test_split()* to split the features and values into separate training and test sets. Use 80% of the original data as a training set, and 20% for testing.


In [ ]:
# 2. Partition into training and testing sets
from IPython.display import display

from sklearn import model_selection

feature_frame = pd.DataFrame(data=bs.data, columns=bs.feature_names)

# Add targets to same data frame so they stay paired with other data during the partition
feature_frame.insert(len(feature_frame.columns), 'TARGET', bs.target)

# train_test_split returns a list of the train-test split of inputs
train_features, test_features = model_selection.train_test_split(feature_frame,
                                               train_size=0.8,
                                               random_state=113) # reproducible outputs

# Using IPython.display allows us to view multiple data frames in the same execution
# and use the pretty IPython format instead of the plain text of print()
print("Training set:")
display(train_features)
print("\n\nTest set:")
display(test_features)

# Now that the set is partitioned, grab the targets into a separate data frame
train_target = pd.DataFrame(data=train_features.iloc[:, -1:])
test_target = pd.DataFrame(data=test_features.iloc[:, -1:])

# And then remove the targets from the feature matrix
train_features = train_features.drop(columns='TARGET')
test_features = test_features.drop(columns='TARGET')

# Experiment 3
Create a *scatterplot* of the training set showing the relationship between the feature LSTAT and the target value MEDV.

***Question***: Does the relationship appear to be linear?

In [ ]:
# 3. Scatter plot of training data
import matplotlib.pyplot as plt

fig_1, ax_1 = plt.subplots() # Obtain a tuple of a figure with one subplot

ax_1.plot(train_features['LSTAT'], train_target, 'c.') # Cyan dots look nice

# Use ax.set to set axis labels and a title
retval = ax_1.set(xlabel="Training LSTAT (%)", ylabel="Training MEDV (1000's USD)",
                 title="Training MEDV vs. Training LSTAT")
# Capture the return value in variable retval so it is not printed by the notebook

## Does the relationship appear to be linear?

The above relationship between the feature LSTAT and the target value MEDV does not quite appear to be linear. If we only examined LSTAT ranges from 10 to 25, it might, but the overall data appears to show a non-linear relationship.

# Experiment 4
With LSTAT as ***X*** and MEDV as ***t***, use *np.linalg.inv()* to compute ***w*** for the training set.

***Question***: What is the equation for MEDV as a linear function of LSTAT?

We will use the following formula to compute the weights:

$$
w = (X^TX)^{-1}X^Tt
$$

(found in the linear_regression_vectors_and_matrices.ipynb notebook)

The equation for MEDV as a linear function of LSTAT is written after the code below.

In [ ]:
# 4. A linear model for MEDV as a function of LSTAT

# Copying Mike's idea, it makes sense to define a function to create the X matrix
def make_poly_X(features, *, max_degree: int = 1, has_ones: bool = False):
    """Creates a copy of the inputted DataFrame, adding a column of 1.0
       values to the left (if necessary). For each column apart from the
       leftmost, higher ordered columns will be inserted just to the
       right of their lower order counterparts.
       
        Intended Use:
       
            Provide features as a Pandas DataFrame. The columns must be
            named and unique for this function to work. The features 
            should be in columns, i.e. each sample is a row.
           
        Parameters:
        
            features:    A DataFrame containing the features.
            
            max_degree:    Maximum degree/exponent that will be generated
            for each column.
            
            has_ones:    Indicate whether the features DataFrame already
            has a leftmost column of 1.0 values.
    """
    
    # If max_degree < 1, return immediately
    if max_degree < 1:
        return None
    
    # Handle Pandas Series objects by creating a frame out of them
    if isinstance(features, pd.Series):
        features = pd.DataFrame(data=features, columns=[features.name])
    # If not a DataFrame at this point, there is an error
    if not isinstance(features, pd.DataFrame):
        return None
    
    # Make a copy to not affect original data
    features = pd.DataFrame(features[features.columns])
        
    # Insert column of ones on the left if not already exists
    if not has_ones:
        features.insert(0, # insert at leftmost column
                       'YINT_CONST', # name of column represents y-intercept constant
                       np.ones_like(features.iloc[:,0]))
        
    # Grab all column names, ignoring the column of ones. This should have names of all features
    feature_columns = features.columns.values[1:]
    # Iterate over each feature
    for feature_name in feature_columns:
        feature_index = features.columns.get_loc(feature_name)
        # For each feature, insert its higher degree columns sequentially
        for exponent in range(2, max_degree+1):
            features.insert(feature_index + exponent - 1, # column index to insert at
                            feature_name + "^" + str(exponent), # column name
                            features[feature_name].pow(exponent)) # Series multiplied
            
    # Return updated features
    return features

# Using our function, create our X matrix for a linear model
lstat_vals = train_features['LSTAT']
lstat_lin = make_poly_X(lstat_vals, max_degree=1, has_ones=False)
display(lstat_lin)

In [ ]:
# Continuing experiment 4, we will define a function to help us with repeated
# calculations of weights in the later questions.

def calc_weights(X_matrix: pd.DataFrame, targets: pd.DataFrame, method="inv", verbose=False):
    """Calculates the weights of a model using the given X matrix and targets.
    
    Intended Use:
    
        Use make_poly_X to get X_matrix as a DataFrame.
        Provide targets as a single column DataFrame.
        
    Parameters:
    
        method:    The method argument determines how this function will
        calculate the weights. Using "inv" will invoke numpy.linalg.inv().
        Using "solve" will invoke numpy.linalg.solve(), and in this case
        the X_matrix must be of full rank. Using "lstsq" will invoke
        numpy.linalg.lstsq().
        
        verbose:    When True, the matrices that are generated during the
        computation of the weights will be printed, and the final weights
        output will be displayed using IPython.display().
    """
    
    # Using linalg.lstsq, for situations where the matrix is not full rank.
    if method == "lstsq":
        solution, residuals, rank, singular_a = np.linalg.lstsq(X_matrix, targets, rcond=None)
        weights = solution
        if verbose:
            print("lstsq residuals:", residuals, sep="\n")
            print("\nlstsq rank:", rank, sep="\n")
            print("\nsingular values:", singular_a, sep="\n")
        
    # The other two methods both use XT_X and XT_t
    else:
        XT_X = np.dot(X_matrix.T, X_matrix) # X.transpose() dot X
        XT_t = np.dot(X_matrix.T, targets) # X.transpose() dot target vector
        if verbose:
            print("XT_X:", XT_X, sep="\n")
            print("\nXT_t:", XT_t, sep="\n")
        
    # Using inverse, according to fcml Chapter 1 appendix, this is not ideal
    # It can become slow and inaccurate
    if method == "inv":
        XT_X_inv = np.linalg.inv(XT_X)
        if verbose:
            print("\nXT_X_inv:", XT_X_inv, sep="\n")
        weights = np.dot(XT_X_inv, XT_t)
    
    # Using linalg.solve, according to fcml Chapter 1 appexndix, is
    # preferable compared to inverse
    elif method == "solve":
        weights = np.linalg.solve(XT_X, XT_t)
        
    # Create a list to rename the rows into w0, w1, ...
    indices = ["w" + str(i) for i in range(len(X_matrix.columns))]
    weights = pd.DataFrame(weights, # put weights into a data frame
                           columns=['WEIGHTS'], # rename the column
                           index=indices)  # rename the rows
    
    if verbose:
        display(weights)
    return pd.DataFrame(weights)
    
lstat_lin_weights = calc_weights(lstat_lin, train_target, method="inv", verbose=True)

## Equation for MEDV as a linear function of LSTAT
Using our training set, the equation for MEDV as a linear function of LSTAT is:

\begin{align}
MEDV = 34.38436369 - (0.93878862 * LSTAT)
\end{align}

# Experiment 5

Use **w** to add a line to your scatter plot from experiment *(3)*.

***Question***: How well does the model appear to fit the training set?


In [ ]:
# 5. Plotting and evaluating the linear MEDV v LSTAT model

# Helper function for creating points for best fit line
def make_x_y(X_column: pd.Series, weights: pd.DataFrame, degree=1):
    """Create a tuple (ndarray, ndarray) where the first ndarray
       contains x-values, and the second ndarray is the corresponding
       y-values. The output is directly compatible with matplotlib's
       plotting functions.
    
    Intended Use:
    
        Provide X_column as a Pandas Series. Use the [] operator
        on a data frame to get a Series. Use calc_weights to get
        weights as a DataFrame. Set degree to match weights.
        
    Parameters:
        
        X_column:    A Pandas Series. Use the DataFrame[] operator
        to get a column from the DataFrame as a Series object.
        
        weights:    A Pandas DataFrame. Use calc_weights() to get
        weights as a DataFrame. Note the degree of the model used.
        
        degree:    The degree of the model. This must match the degree
        used during calc_weights.
    """
    
    # Generate uniform values across the range of [min(X_column), max(X_column)]
    x_uniform = np.linspace(X_column.min(), X_column.max(), X_column.count())

    # Put the new x values in a data frame to make the X matrix
    X_matrix = pd.DataFrame(data=x_uniform, columns=[X_column.name])
    X_matrix = make_poly_X(X_matrix, max_degree=degree, has_ones=False)
    y_predictions = np.dot(X_matrix, weights)
    return (x_uniform, y_predictions)


lstat_linear_x, lstat_linear_y = make_x_y(lstat_vals, lstat_lin_weights, degree=1)

fig_2, ax_2 = plt.subplots()
# Scatter actual training data
ax_2.plot(train_features['LSTAT'], train_target, 'c.')
# Create line of best fit using points that are uniformly spaced in the x-axis
ax_2.plot(lstat_linear_x, lstat_linear_y, 'r-', label="Linear Least Squares")
ax_2.legend()
retval = ax_2.set(xlabel="Training LSTAT (%)", ylabel="Training MEDV (1000's USD)",
                 title="Training MEDV vs. Training LSTAT")

# How well does the model appear to fit the training set?
5. The linear model of MEDV as a function of LSTAT does not appear to be a good fit for the data.

# Experiment 6
Use **w** to find the response for each value of the LSTAT attribute in the test set, then compute the test *MSE L* for the model.

In [ ]:
# 6. Computing MSE for the linear MEDV v LSTAT model on the test set

# Create helper function to calculate MSE
def calc_MSE(X_matrix: pd.DataFrame, weights: pd.DataFrame, targets: pd.DataFrame, verbose=False):
    """Calculate the mean squared error using the given X_matrix, weights,
       and targets.
       
    Intended Use:
    
        Use make_poly_X to get X_matrix as a DataFrame.
        Use calc_weights to get weights as a DataFrame.
        Provide targets as a DataFrame.
        
    Parameters:
    
        X_matrix:    DataFrame containing the features with a column of
        ones and higher ordered columns if applicable. This X_matrix
        must have the same max_degree as what was used to calculate the
        weights.
        
        weights:    DataFrame of weights (one column) that will be used
        on the X_matrix to get predictions.
        
        targets:    Target values to compare predictions with.
        
        verbose:    If True, displays the residuals as a DataFrame.
    """
    # This becomes a numpy ndarray
    predictions = np.dot(X_matrix, weights) # X * w = y(hat)
    
    # This becomes a DataFrame again
    residuals = targets - predictions # Residuals vector (observed - predicted)
    residuals.columns = ['RESID']
    if verbose:
        display(residuals)
    
    # This becomes a numpy ndarray once more, with shape (1, 1)
    sum_squared_error = np.dot(residuals.T, residuals) # dotting resids with self
    # We need to grab the scalar value inside
    sum_squared_error = sum_squared_error[0][0]
    
    # Now divide sum sq. err. w/ the number of values to get mean sq. err. (MSE)
    return sum_squared_error / len(predictions)
    
# Grab the LSTAT values for the test set
lstat_test_vals = test_features['LSTAT']

# Create the X matrix for the test LSTAT
lstat_test_lin = make_poly_X(lstat_test_vals, max_degree=1, has_ones=False)

lstat_lin_MSE = calc_MSE(lstat_test_lin, lstat_lin_weights, test_target)
print("Testing MSE for LSTAT linear model:", lstat_lin_MSE)

# Graphing the result of prediction on test set (optional)
# lstat_lin_test_x, lstat_lin_test_y = make_x_y(lstat_test_vals, lstat_lin_weights, degree=1)
# fig_3, ax_3 = plt.subplots()
# # Scatter actual testing data
# ax_3.plot(lstat_test_vals, test_target, 'c.')
# # Create line of best fit using points that are uniformly spaced in the x-axis
# ax_3.plot(lstat_lin_test_x, lstat_lin_test_y, 'r-', label="Prediction using weights from training")
# ax_3.legend()
# retval = ax_3.set(xlabel="Test LSTAT (%)", ylabel="Test MEDV (1000's USD)",
#                  title="Testing MEDV vs. Testing LSTAT")

##  MSE For Linear Model

6. The testing MSE for the LSTAT linear model was 39.95751318608688. The predictions are made using the weights derived from the training set, and are compared with the actual test data targets. The linear model still appears to be a poor predictor for MEDV.

# Experiment 7
Now add an *x 2* column to LSTAT’s *x* column in the training set, then repeat experiments *(4)* , *(5)*, and *(6)* for MEDV as a quadratic function of LSTAT.

***Question***: Does the quadratic polynomial do a better job of predicting the values in the test set?

## Performing Experiment 4 on the New Quadratic Matrix
With LSTAT as ***X*** and MEDV as ***t***, use *np.linalg.inv()* to compute ***w*** for the training set.

***Question***: What is the equation for MEDV as a ***QUADRATIC*** function of LSTAT?

In [ ]:
# 7 (4). Quadratic LSTAT model

lstat_quad = make_poly_X(lstat_vals, max_degree=2, has_ones=False)
display(lstat_quad)

# We use the previously defined function, using the linalg.inv method
lstat_quad_weights = calc_weights(lstat_quad, train_target, method="inv")
display(lstat_quad_weights)

### What is the equation for MEDV as a ***QUADRATIC*** function of LSTAT?

The equation for MEDV using the quadratic model is:

\begin{aligned}
\text{MEDV} = 41.8618826 - 2.163404190 * \text{LSTAT} + 0.0382868299 * \text{LSTAT}^2
\end{aligned}

## Performing Experiment 5 on the new Quadratic Matrix
Use ***w*** to add a line to your scatter plot from experiment *(3)*.

***Question***: How well does the model appear to fit the training set?

In [ ]:
# 7 (5) Adding best fit line to scatter

# Use our function to create points for the best-fit curve
# Without this the points would be out of order and curve goes berserk
lstat_quad_x, lstat_quad_y = make_x_y(lstat_vals, lstat_quad_weights, degree=2)

# Plot best fit quad model on training data
fig_4, ax_4 = plt.subplots()
# Plot the actual training data
ax_4.plot(lstat_vals, train_target, 'c.')
# Draw the quadratic best fit curve
ax_4.plot(lstat_quad_x, lstat_quad_y, 'r-', label="Quadratic model\n(least squares)")
ax_4.legend()
retval = ax_4.set(xlabel="Test LSTAT (%)", ylabel="Test MEDV (1000's USD)",
                 title="Testing MEDV vs. Testing LSTAT")

### How well does the model appear to fit the training set?
7. (5) Visually, the model appears to fit the training data a lot better than the linear model.

## Performing Experiment 6 on the New Quadratic Matrix
Use ***w*** to find the response for each value of the LSTAT attribute in the test set, then
compute the test *MSE L* for the model.

In [ ]:
# 7 (6) Computing MSE of the quadratic model

# Recall that the LSTAT values for the test set are in l_stat_test_vals

# Create the quad X matrix for the test LSTAT
lstat_test_quad = make_poly_X(lstat_test_vals, max_degree=2, has_ones=False)

lstat_quad_MSE = calc_MSE(lstat_test_quad, lstat_quad_weights, test_target, verbose=True)
print("Testing MSE for LSTAT quadratic model:", lstat_quad_MSE)

## Does the quadratic polynomial do a better job of predicting the values in the test set?

7. (6) The test MSE for the LSTAT quadratic model is lower than that of the linear model.

Q: Does the quadratic polynomial do a better job of predicting the values in the test set?

A: Yes, the quadratic polynomial has a lower testing MSE; therefore it predicts the values of the test set better.

# Experiment 8
Repeat experiment *(4)* with all 13 input features as ***X*** and using *np.linalg.lstsq()*. (See the Appendix to *Linear regression in vector and matrix format* for details of why we need to switch away from *np.linalg.inv()*, and the notes for *np.linalg.solve()* for why we shouldn’t use that either.)

***Question***: Does adding additional features improve the performance on the test set compared to using only LSTAT?

In [ ]:
# 8. All 13 features, linear model

# Recall that the training features data frame is: train_features

# Create X matrix for 13 features
all_feat_lin = make_poly_X(train_features, max_degree=1, has_ones=False)

# (XT X) w = (XT t)
# Here, (XT X) is a square matrix, and (XT t) is a vector of dependent variables
# Aw = b
# np.linalg.solve(A, b) gives w

# Later edit: the assignment prompt changed and we implemented calc_weights
# to also allow method="lstsq". This will use linalg.lstsq() in the calculation.

afl_weights = calc_weights(all_feat_lin, train_target, method="lstsq", verbose=True)

8. The equation for MEDV using all 13 features is:

\begin{aligned}
\text{MEDV}\ =\ & 33.5486261 \\
& - 0.114331280(\text{CRIM}) +  0.0330399664(\text{ZN}) + 0.0219911151(\text{INDUS}) \\
& + 1.93047806(\text{CHAS}) - 15.3459876(\text{NOX}) +  4.11678898(\text{RM}) \\
& - 0.00520475977(\text{AGE}) - 1.26111638(\text{DIS}) + 0.352665352(\text{RAD}) \\
& - 0.0137375084(\text{TAX}) - 1.01521476(\text{PTRATIO}) + 0.00998692962(\text{B}) \\
& -0.490950094(\text{LSTAT})
\end{aligned}

In [ ]:
# Make predictions on test sest

# Create X matrix with test features
all_feat_lin_test = make_poly_X(test_features[test_features.columns.values], max_degree=1, has_ones=False)

afl_MSE = calc_MSE(all_feat_lin_test, afl_weights, test_target, verbose=False)
print("Testing MSE for 13 feature linear model:", afl_MSE)
print("Recall the testing MSE for LSTAT linear model:", lstat_lin_MSE)

## Improvement with Additional Features?
*Does adding additional features improve the performance on the test set compared to using only LSTAT?*

8. Comparing the linear models, accuracy seems to improve with 13 features, as opposed to just LSTAT.

# Experiment 9
Now add $x^2$ columns for all 13 features, and repeat experiment *(8)*.

***Question***: Does adding quadratic features improve the performance on the test set compared to using only linear features?

In [ ]:
# 9. Quadratic model with all features

# Recall that the training features data frame is: train_features

# Create X matrix
all_feat_quad = make_poly_X(train_features[train_features.columns.values], max_degree=2, has_ones=False)
# Note that the CHAS (Charles River flag variable) is still the same after squaring,
# because its values are either 0 or 1. This would make the matrix no longer linearly
# independent, which has two problems:
# 1. The dependent column adds no expressive value to our model. It is redundant.
# 2. The dependence means there are infinite solutions and np.linalg.solve() won't work.
#    Later edit: we are no longer using linalg.solve
# Manually remove the CHAS^2 column to fix this.
all_feat_quad = all_feat_quad.drop(columns='CHAS^2')

afq_weights = calc_weights(all_feat_quad, train_target, method="lstsq")
display(afq_weights.T)

In [ ]:
# To compare performance, compute the test MSE for the all features quadratic model
afq_test = make_poly_X(test_features[test_features.columns.values], max_degree=2, has_ones=False)

# Once again make sure to drop CHAS^2
afq_test = afq_test.drop(columns='CHAS^2')

afq_MSE = calc_MSE(afq_test, afq_weights, test_target)
print("Testing MSE for 13 feature quadratic model:", afq_MSE)
print("Recall the testing MSE for 13 feature linear model:", afl_MSE)

## Improved Performance With Quadratic Features?
Q: Does adding quadratic features improve the performance on the test set compared to using only linear features?

A: Yes. It appears that the testing MSE has dropped significantly using the quadratic model.

# Experiment 10
Compute the training MSE for experiments *(8)* and *(9)* and compare it to the test MSE.

***Question***: What explains the difference?

In [ ]:
# 10. Computing training MSE for both linear and quadratic 13-feature models.

# Training MSE for 13 feature linear
# Recall that the X matrix for the 13 feature linear model is: all_feat_lin
# Recall that the weight vector for the 13 feature linear model is: afl_weights
afl_train_MSE = calc_MSE(all_feat_lin, afl_weights, train_target)

# Training MSE for 13 feature quadratic
# Recall that the X matrix for the 13 feature quadratic model is: all_feat_quad
# Recall that the weight vector for the 13 feature quadratic model is: afq_weights
afq_train_MSE = calc_MSE(all_feat_quad, afq_weights, train_target)

print("Training MSE for 13 feature linear:", afl_train_MSE)
print("Testing MSE for 13 feature linear:", afl_MSE, end="\n\n")

print("Training MSE for 13 feature quadratic:", afq_train_MSE)
print("Testing MSE for 13 feature quadratic:", afq_MSE)

## What explains the difference?: Answer
10. Although the testing MSE went up compared to the training MSE during the use of our 13-feature linear model, the linear model is the simplest model we can have, so we cannot be overfitting.

  It appears our linear model is an underfit. Our quadratic model produced a much better training MSE, and the improvement persisted during our testing phase. This suggests that the true behavior is more closely modeled by the quadratic model than by the linear model.



# Experiment 11
Repeat experiments *(9)* and *(10)*, adding *x^3* columns in addition to the existing *x* and *x^2* columns for each feature. Does the cubic polynomial do a better job of predicting the values in the training set?

***Question***: Does it do a better job of predicting the values in the test set?

In [ ]:
# 11. Going to cubic

# afc = all features cubic
afc_train_X = make_poly_X(train_features[train_features.columns.values], max_degree=3, has_ones=False)
afc_test_X = make_poly_X(test_features[test_features.columns.values], max_degree=3, has_ones=False)

# As with the quadratic model, we have to remove higher order of the CHAS column
# to avoid linear dependence.
afc_train_X = afc_train_X.drop(columns=['CHAS^2', 'CHAS^3'])
afc_test_X = afc_test_X.drop(columns=['CHAS^2', 'CHAS^3'])

display(afc_train_X) # Take a look at this big frame

# Calculate weights using training set
afc_weights = calc_weights(afc_train_X, train_target, method="lstsq")

# Calculate MSE for training and test sets
afc_train_MSE = calc_MSE(afc_train_X, afc_weights, train_target)
afc_test_MSE = calc_MSE(afc_test_X, afc_weights, test_target)

print("Training MSE for 13 feature cubic model:", afc_train_MSE)
print("Testing MSE for 13 feature cubic model:", afc_test_MSE)

# Improved Performance With Cubic Features?
*Does adding **CUBIC** features improve the performance on the test set compared to using only **LINEAR OR QUADRATIC** features?* Does the cubic polynomial do a better job of predicting the values in the training set? Does it do a better job of predicting the values in the test set? Compute the MSE for experiments 8 and 9 and compare it to the test MSE. What explains the difference?*

11. The cubic polynomial (13 feature model) predicts the training set better than its quadratic and linear counterparts. It also predicts the test set better than the quadratic and linear models. The training MSE for the cubic model was lower than the testing MSE. We can infer from this data that the cubic model is a better fit, and more closely models the true relationship between MEDV and the other 13 features.